In [2]:
!pip install --quiet git+https://github.com/huggingface/transformers sentencepiece
!pip install torchaudio

In [3]:
from transformers import SeamlessM4TModel, AutoProcessor
import torch


model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-medium")


device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)


processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-medium")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
import torchaudio
from tqdm import tqdm

# Function to segment the audio file
def segment_audio(audio_waveform, sample_rate, segment_length=10, overlap=1):
    # Calculate the number of samples per segment
    num_samples_per_segment = sample_rate * segment_length
    overlap_samples = sample_rate * overlap

    # Segment the audio
    segments = []
    start = 0
    while start < audio_waveform.shape[1]:
        end = start + num_samples_per_segment
        segments.append(audio_waveform[:, start:end])
        start = end - overlap_samples  # ensure overlap
    return segments

# Load the audio file
waveform, sample_rate = torchaudio.load('/kaggle/input/hamidi-voices/output_10m.wav')

# Segment the audio file
segments = segment_audio(waveform, sample_rate)

# Initialize the text variable
transcribed_text = ""

# Process each segment
for segment in tqdm(segments, desc="Processing Segments"):
    # Resample if necessary
    if sample_rate != model.config.sampling_rate:
        segment = torchaudio.functional.resample(segment, orig_freq=sample_rate, new_freq=model.config.sampling_rate)

    # Process audio segment
    audio_inputs = processor(audios=segment.squeeze().numpy(), sampling_rate=model.config.sampling_rate, return_tensors="pt").to(device)
    
    try:
        # Generate output tokens
        output_tokens = model.generate(**audio_inputs, tgt_lang="pes", generate_speech=False)
        segment_text = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)

        # Append to the full transcription
        transcribed_text += " " + segment_text
    except Exception as e:
        print(f"Error processing segment: {e}")

# Print the complete transcribed text
print(f"Transcribed Text: {transcribed_text}")


Processing Segments: 100%|██████████| 67/67 [00:38<00:00,  1.73it/s]

Transcribed Text:  این انفراسترچر احتمالا بیشتر میشه تا اون صنعت یا ورتیکال خاص که داشتینش کار میکردیم. روشن چی میگیم؟ تو تو تو چیزی ساخته بدر؟ بدر چی؟ اینم: اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم که اینم آفاین؟ میشه کمک کنیم بهترش بگی؟ خله ای توی دنیا وجود داره؟ آها، اون خله رو میتونم پر کنم و اگه خله خودم رو به ما پر میکنم، شاید خله های دیگه رو هم به ما پر میکنن. اگه محدودش نکنم به خودتون، من حالا آخرش میخواستم اینو بگم آقای حبیبی همیشه جلو جلو میده واقعاً. [مذکر: من یه لانه از یه موزوی گاز دارم که میخوام بگم] یه موضوع دیگه هست که پول بگیری یا پول نگیری یا حرفیشون؟ [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: [مذکر: این دوستان ریموت فقط صدای خنده ها رو میشنوند و نمیدونن که اینجا چی داره گفته میشن. خب. وا من فهمیدم اصلا نفهمیدم شما چیکار کردید و چی گفتی مثل همین، وگرنه خیل